In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from tqdm import tqdm_notebook 
import seaborn as sns
import time
from IPython.display import HTML
from sklearn.preprocessing import StandardScaler


from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_blobs

In [2]:
class FF_MultiClass_WeightVectorised:
  
  def __init__(self):
    self.W1 = np.random.randn(4,2)
    self.W2 = np.random.randn(2,3)
    self.B1 = np.zeros((1,2))
    self.B2 = np.zeros((1,3))
  
  def sigmoid(self, x):
    return 1.0/(1.0 + np.exp(-x))
  
  def softmax(self, x):
    exps = np.exp(x)
    return exps / np.sum(exps)
  
  def forward_pass(self, x):
    x = x.reshape(1, -1)
    self.A1 = np.matmul(x,self.W1) + self.B1
    self.H1 = self.sigmoid(self.A1)
    self.A2 = np.matmul(self.H1, self.W2) + self.B2
    self.H2 = self.softmax(self.A2) 
    return self.H2
    
  def grad_sigmoid(self, x):
    return x*(1-x) 
  
  def grad(self, x, y):
    self.forward_pass(x)
    x = x.reshape(1, -1) 
    y = y.reshape(1, -1) 
    
    self.dA2 = self.H2 - y 
    
    self.dW2 = np.matmul(self.H1.T, self.dA2) 
    self.dB2 = self.dA2 # (1, 4)
    self.dH1 = np.matmul(self.dA2, self.W2.T) 
    self.dA1 = np.multiply(self.dH1, self.grad_sigmoid(self.H1))
    
    self.dW1 = np.matmul(x.T, self.dA1)
    self.dB1 = self.dA1

  
  def fit(self, X, Y, epochs=1, learning_rate=1, display_loss=False):
      
    if display_loss:
      loss = {}
    
    for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
      dW1 = np.zeros((4,2))
      dW2 = np.zeros((2,3))
      dB1 = np.zeros((1,2))
      dB2 = np.zeros((1,3))
      for x, y in zip(X, Y):
        self.grad(x, y)
        dW1 += self.dW1
        dW2 += self.dW2
        dB1 += self.dB1
        dB2 += self.dB2  
        
      m = X.shape[0]
      self.W2 -= learning_rate * (dW2/m)
      self.B2 -= learning_rate * (dB2/m)
      self.W1 -= learning_rate * (dW1/m)
      self.B1 -= learning_rate * (dB1/m)

      if display_loss:
        Y_pred = self.predict(X)
        loss[i] = log_loss(np.argmax(Y, axis=1), Y_pred)
        
    
    if display_loss:
      plt.plot(loss.values())
      plt.xlabel('Epochs')
      plt.ylabel('Log Loss')
      plt.show()
      
  def predict(self, X):
    Y_pred = []
    for x in X:
      y_pred = self.forward_pass(x)
      Y_pred.append(y_pred)
    return np.array(Y_pred).squeeze()

In [3]:
df = pd.read_csv("data-set.csv")
df.head()

,Cgpa,Age,IETLS,Publications,University
0,3.50,28,5.4,6,California
1,3.79,25,3.0,1,Toronto
2,3.90,22,8.5,20,Stanford
3,3.58,27,5.3,7,California
4,3.78,26,3.4,2,Toronto


In [4]:
y = df['University']
x = df.drop(['University'], axis=1)

In [5]:
X_train, X_val, Y_train, Y_val = train_test_split(x, y, random_state=0)
print(X_train.shape, X_val.shape, y.shape)

(36, 4) (12, 4) (48,)


In [6]:
sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_val=sc.transform(X_val)
print(X_train.shape, X_val.shape, y.shape)

(36, 4) (12, 4) (48,)


In [7]:
enc = OneHotEncoder()
y_OH_train = enc.fit_transform(np.expand_dims(Y_train,1)).toarray()
y_OH_val = enc.fit_transform(np.expand_dims(Y_val,1)).toarray()
print(y_OH_train.shape, y_OH_val.shape)
y_OH_train

(36, 3) (12, 3)


array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [8]:
weight_vectorised = FF_MultiClass_WeightVectorised()
weight_vectorised.fit(X_train,y_OH_train,epochs=1000,learning_rate=1)

C:\Users\ASPHAL~1\AppData\Local\Temp/ipykernel_3236/3869363629.py:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):


  0%|          | 0/1000 [00:00<?, ?epoch/s]

In [9]:
Y_pred_train = weight_vectorised.predict(X_train)
Y_pred_train = np.argmax(Y_pred_train,1)

Y_pred_val = weight_vectorised.predict(X_val)
Y_pred_val = np.argmax(Y_pred_val,1)

new_y_OH_val = np.argmax(y_OH_val,1)
new_y_OH_train = np.argmax(y_OH_train,1)


accuracy_train = accuracy_score(Y_pred_train, new_y_OH_train)
accuracy_val = accuracy_score(Y_pred_val, new_y_OH_val)

print("Training accuracy", round(accuracy_train, 2))
print("Validation accuracy", round(accuracy_val, 2))

Training accuracy 1.0
Validation accuracy 1.0


In [11]:
qry = np.argmax(FF_MultiClass_WeightVectorised.predict([[3,12,9,10]]),1)
qry

TypeError: predict() missing 1 required positional argument: 'X'